In [32]:
# encoding:utf-8
# 产生核对的数据
import datetime
import pymongo
import time
from math import log,sqrt,exp

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats

#并行计算
import ipyparallel as ipp

#多处理
import multiprocessing as mp

#多线程
import numexpr as ne

#动态编译
import numba as nb
#静态编译
#############

c = ipp.Client()

print(c.ids)
    
def bsm_call(S,K,T,r,sigma):
    #S=float(S)
    d1=(log(S/K)+0.5*sigma**2*T)/(sigma*sqrt(T))
    d2=(log(S/K)-0.5*sigma**2*T)/(sigma*sqrt(T))
    value=(S*stats.norm.cdf(d1,0.0,1.0)-K*exp(-r*T)*stats.norm.cdf(d2,0.0,1.0))
    return value


def bsm_put(S,K,T,r,sigma):
    #S=float(S)
    d1=(log(S/K)+0.5*sigma**2*T)/(sigma*sqrt(T))
    d2=(log(S/K)-0.5*sigma**2*T)/(sigma*sqrt(T))
    value=(S*(stats.norm.cdf(d1,0.0,1.0)-1)-K*exp(-r*T)*(stats.norm.cdf(d2,0.0,1.0)-1))
    return value


def bsm_vega(S,K,T,r,sigma):
    #S=float(S)
    d1=(log(S/K)+0.5*sigma**2*T)/(sigma*sqrt(T))
    vega=S*stats.norm.cdf(d1,0.0,1.0)*sqrt(T)
    return vega


def bsm_call_imp_vol(S,K,T,r,C,sigma_est,it=200,control=0.01):
    i=0    
    while i<=it:
        sigma_est-=((bsm_call(S,K,T,r,sigma_est)-C)/bsm_vega(S,K,T,r,sigma_est)/100)
        if abs(bsm_call(S,K,T,r,sigma_est)-C)<control:
            return sigma_est
        else:
            i+=1
    return sigma_est


def bsm_put_imp_vol(S,K,T,r,P,sigma_est,it=200,control=0.01):
    i=0    
    while i<=it:
        sigma_est-=((bsm_put(S,K,T,r,sigma_est)-P)/bsm_vega(S,K,T,r,sigma_est)/100)
        if abs(bsm_put(S,K,T,r,sigma_est)-P)<control:
            return sigma_est
        else:
            i+=1
    return sigma_est

def calculate_vol(row):
    vol = bsm_call_imp_vol(S=row['askPrice1'], K=3000, T=0.2, r=0.03, C=row['askPrice1_c'], sigma_est=0.2)
    return {'datatime':row['datetime'], 'volatility': vol}

#创建服务器
default_client = pymongo.MongoClient('localhost', 27017)
#选择数据库
space_db = default_client['parity_space']
datetime1 = datetime.datetime(2017, 5, 16, 9, 0, 0)
datetime2 = datetime.datetime(2017, 6, 6, 10, 0, 0)
# projection是list形式
space_query = {
    'datetime':{'$gt': datetime1, '$lt':datetime2},
    'askPrice1':{'$gt': 0},
    'askPrice1_c':{'$gt': 0},
    'askPrice1_p':{'$gt': 0},
    'bidPrice1':{'$gt': 0},
    'bidPrice1_c':{'$gt': 0},
    'bidPrice1_p':{'$gt': 0},
    
}
start_time = time.time()
#选择collection和document
docs = space_db['Paritym17093000Strategy'].find(space_query).sort([('datetime', pymongo.ASCENDING)])

#多处理：失败
#pool = mp.Pool(processes=10)
#vol_list = pool.map(calculate_vol, list(docs))
#异步集群：语法错误
#vol_list = [c[:].apply_sync(calculate_vol, doc) for doc in docs]

last_vol = 0.2
vol_list = []
for row in docs:
    vol = bsm_call_imp_vol(S=row['askPrice1'], K=2600, T=0.2, r=0.04, C=row['askPrice1_c'], sigma_est=last_vol)
    last_vol = vol
    vol_list.append({'datatime':row['datetime'], 'volatility': vol})

#vol_list = [calculate_vol(x) for x in docs]
#保存隐含波动率结果的df
vol_dataframe = pd.DataFrame(vol_list)

vol_dataframe.volatility.plot()
end_time = time.time()
print("use time: %r" % (end_time-start_time))
plt.show()

[0, 1, 2, 3]


D:\Anaconda2\envs\volatility\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in double_scalars
D:\Anaconda2\envs\volatility\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
D:\Anaconda2\envs\volatility\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars
D:\Anaconda2\envs\volatility\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
D:\Anaconda2\envs\volatility\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
D:\Anaconda2\envs\volatility\lib\site-packages\scipy\stats\_distn_infrastructure.py:1735: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
D:\Anaconda2\envs\volatility\lib\site-packages\ipykernel_launche

CursorNotFound: Cursor not found, cursor id: 2449359893388

In [22]:
#encoding:utf-8
# encoding:utf-8
# 产生核对的数据
import datetime
import pymongo
import time
import random
from math import log,sqrt,exp,cos

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats

#并行计算
import ipyparallel as ipp

#多处理
import multiprocessing as mp

#多线程
import numexpr as ne

#动态编译
import numba as nb
#静态编译

@nb.jit
def f_py(I):
    res = 0
    for i in range(I):
        for j in range(I):
            res += cos(log(1))
    return res

start_time = time.time()

pool = mp.Pool(processes = 10)
result_list = map(f_py, range(500))

#I, J = 5000, 5000
#result = f_py(I, J)
end_time = time.time()
print("use time: %r" % (end_time - start_time))
print result_list
print([(1,2)] *10)

use time: 0.12400007247924805
[0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0, 100.0, 121.0, 144.0, 169.0, 196.0, 225.0, 256.0, 289.0, 324.0, 361.0, 400.0, 441.0, 484.0, 529.0, 576.0, 625.0, 676.0, 729.0, 784.0, 841.0, 900.0, 961.0, 1024.0, 1089.0, 1156.0, 1225.0, 1296.0, 1369.0, 1444.0, 1521.0, 1600.0, 1681.0, 1764.0, 1849.0, 1936.0, 2025.0, 2116.0, 2209.0, 2304.0, 2401.0, 2500.0, 2601.0, 2704.0, 2809.0, 2916.0, 3025.0, 3136.0, 3249.0, 3364.0, 3481.0, 3600.0, 3721.0, 3844.0, 3969.0, 4096.0, 4225.0, 4356.0, 4489.0, 4624.0, 4761.0, 4900.0, 5041.0, 5184.0, 5329.0, 5476.0, 5625.0, 5776.0, 5929.0, 6084.0, 6241.0, 6400.0, 6561.0, 6724.0, 6889.0, 7056.0, 7225.0, 7396.0, 7569.0, 7744.0, 7921.0, 8100.0, 8281.0, 8464.0, 8649.0, 8836.0, 9025.0, 9216.0, 9409.0, 9604.0, 9801.0, 10000.0, 10201.0, 10404.0, 10609.0, 10816.0, 11025.0, 11236.0, 11449.0, 11664.0, 11881.0, 12100.0, 12321.0, 12544.0, 12769.0, 12996.0, 13225.0, 13456.0, 13689.0, 13924.0, 14161.0, 14400.0, 14641.0, 14884.0, 15129.0,

In [15]:
def add_two(x, y):
    print(x+y)
add_two(*(1,2))

3
